In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras import backend as K
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization,LeakyReLU
from keras.layers.merge import concatenate
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import glob

HEIGHT=75
WIDTH=75

train_path = r"C:\Users\voolo\Image_colorization_autoencoder\train\color"
val_path = r"C:\Users\voolo\Image_colorization_autoencoder\valid\color"
test_path = r"C:\Users\voolo\Image_colorization_autoencoder\test\color"

def ExtractInput(path):
    X_img=[]
    y_img=[]
    for name in glob.glob(train_path+'\*.jpg'):
        try:
            img = cv2.imread(name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
            img = img.astype(np.float32)
            img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
            #Convert the rgb values of the input image to the range of 0 to 1
            #1.0/255 indicates that we are using a 24-bit RGB color space.
            #It means that we are using numbers between 0–255 for each color channel
            #img_lab = 1.0/225*img_lab
            # resize the lightness channel to network input size 
            img_lab_rs = cv2.resize(img_lab, (WIDTH, HEIGHT)) # resize image to network input size
            img_l = img_lab_rs[:,:,0] # pull out L channel
            #img_l -= 50 # subtract 50 for mean-centering
            img_ab = img_lab_rs[:,:,1:]#Extracting the ab channel
            img_ab = img_ab/128
            #The true color values range between -128 and 128. This is the default interval 
            #in the Lab color space. By dividing them by 128, they too fall within the -1 to 1 interval.
            X_img.append(img_l)
            y_img.append(img_ab)
        except:
            pass
    X_img = np.array(X_img)
    y_img = np.array(y_img)
    
    return X_img,y_img

In [7]:
L_train,ab_train = ExtractInput(train_path)
L_val,ab_val = ExtractInput(val_path)
L_test,ab_test = ExtractInput(test_path)

In [8]:
print(L_train.shape, ab_train.shape,L_val.shape,ab_val.shape,L_test.shape,ab_test.shape)

(25670, 75, 75) (25670, 75, 75, 2) (25670, 75, 75) (25670, 75, 75, 2) (25670, 75, 75) (25670, 75, 75, 2)
